In [ ]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:

# ============================
# 1. CARREGAR DATASET
# ============================
df = pd.read_csv("/content/dataset-1000-com-preco-promocional-e-renovacao-estoque.csv")
df["DATA_EVENTO"] = pd.to_datetime(df["DATA_EVENTO"])
df = df.sort_values(["ID_PRODUTO", "DATA_EVENTO"])

In [ ]:
# ============================
# 2. CALCULAR CONSUMO (TARGET)
# ============================
df["estoque_amanha"] = df.groupby("ID_PRODUTO")["QUANTIDADE_ESTOQUE"].shift(-1)
df["consumo"] = df["QUANTIDADE_ESTOQUE"] - df["estoque_amanha"]

# Clipa negativos (reposição)
df["consumo"] = df["consumo"].clip(lower=0)

df = df.dropna(subset=["consumo"])

In [ ]:
# ============================
# 3. FEATURES
# ============================
df["ano"] = df["DATA_EVENTO"].dt.year
df["mes"] = df["DATA_EVENTO"].dt.month
df["dia"] = df["DATA_EVENTO"].dt.day
df["dia_semana"] = df["DATA_EVENTO"].dt.weekday

# Lags do consumo
df["consumo_lag1"] = df.groupby("ID_PRODUTO")["consumo"].shift(1)
df["consumo_lag7"] = df.groupby("ID_PRODUTO")["consumo"].shift(7)
df["consumo_mm7"] = (
    df.groupby("ID_PRODUTO")["consumo"]
    .rolling(7)
    .mean()
    .reset_index(level=0, drop=True)
)
# Estoque
df["estoque_lag1"] = df.groupby("ID_PRODUTO")["QUANTIDADE_ESTOQUE"].shift(1)

# Preço
df["preco_lag1"] = df.groupby("ID_PRODUTO")["PRECO"].shift(1)
df["delta_preco"] = df["PRECO"] - df["preco_lag1"]

df = df.fillna(0)

In [ ]:
# ============================
# 4. SPLIT
# ============================
features = [
    "PRECO", "FLAG_PROMOCAO",
    "consumo_lag1", "consumo_lag7", "consumo_mm7",
    "estoque_lag1",
    "preco_lag1", "delta_preco",
    "ano", "mes", "dia", "dia_semana"
]

X = df[features]
y = df["consumo"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [ ]:
# ============================
# 5. TREINO
# ============================
model = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6
)

model.fit(X_train, y_train)

In [ ]:
# ============================
# 6. AVALIAÇÃO
# ============================
preds = model.predict(X_test)
media = mean_absolute_error(y_test, preds)

print(f"Media do consumo: {media:.2f}")
print(f"avaliação: {preds}" )
